In [ ]:
!pip install -q "transformers==4.40.0" "tensorflow==2.11.0" "tensorflow-text==2.11.0" scikit-learn xgboost joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 2.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement tensorflow==2.11.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.11.0


**Importing Libraries**

In [ ]:
import os, random, re, math, time
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
import joblib

import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
from transformers import (DistilBertTokenizer, TFDistilBertForSequenceClassification, BertTokenizer, TFBertModel, create_optimizer)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow version:", tf.__version__)
tf.keras.mixed_precision.set_global_policy('mixed_float16')
AUTO = tf.data.AUTOTUNE
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 6
SAMPLE_FRAC = 1.0

TensorFlow version: 2.19.0


**Preprocessing**

In [ ]:
df = pd.read_csv("/content/labeled_data.csv")
print("Columns:", df.columns.tolist())

text_col, label_col = None, None
for c in df.columns:
    if c.lower() in ("text","tweet","sentence","comment"):
        text_col = c; break
if text_col is None:
    for c in df.columns:
        if df[c].dtype == object:
            text_col = c; break

for c in df.columns:
    if c.lower() in ("label","class","target","y"):
        label_col = c; break
if label_col is None:
    for c in df.columns:
        if np.issubdtype(df[c].dtype, np.number) and df[c].nunique() <= 10:
            label_col = c; break

if text_col is None or label_col is None:
    raise RuntimeError("Could not infer text or label columns. Inspect df.columns and set manually.")

df = df[[text_col, label_col]].rename(columns={text_col: "text", label_col: "label"}).dropna().reset_index(drop=True)
df['label'] = df['label'].astype(int)
print("Total examples:", len(df))
print(df['label'].value_counts())
def clean_tf(t): return re.sub(r"\s+", " ", str(t)).strip()
def clean_cl(t):
    s = str(t).lower()
    s = re.sub(r"http\S+", " URL ", s)
    s = re.sub(r"@\w+", " USER ", s)
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df['text_tf'] = df['text'].apply(clean_tf)
df['text_cl'] = df['text'].apply(clean_cl)

train_df, test_df = train_test_split(df, test_size=0.20, stratify=df['label'], random_state=SEED)
train_df, val_df = train_test_split(train_df, test_size=0.125, stratify=train_df['label'], random_state=SEED)
print("Sizes (train/val/test):", len(train_df), len(val_df), len(test_df))

Columns: ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet']
Total examples: 24783
label
1    19190
2     4163
0     1430
Name: count, dtype: int64
Sizes (train/val/test): 17347 2479 4957


**Result Matrix**

In [ ]:
def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    print("Accuracy:", acc, "Macro-F1:", f1)
    print(classification_report(y_true, y_pred))
    return acc, f1


# **BiLSTM**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer_lstm = Tokenizer(oov_token="<OOV>")
tokenizer_lstm.fit_on_texts(train_df['text_cl'])
vocab_size = len(tokenizer_lstm.word_index) + 1
MAX_LEN_LSTM = 50

def to_seq(texts):
    return pad_sequences(tokenizer_lstm.texts_to_sequences(texts), maxlen=MAX_LEN_LSTM, padding='post')

train_seq = to_seq(train_df['text_cl'])
val_seq   = to_seq(val_df['text_cl'])
test_seq  = to_seq(test_df['text_cl'])

y_train = train_df['label'].values
y_val   = val_df['label'].values
y_test  = test_df['label'].values

class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self): super().__init__()
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1],1), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(1,), initializer='zeros', trainable=True)
    def call(self, inputs):
        score = tf.nn.tanh(tf.tensordot(inputs, self.w, axes=1) + self.b)
        weights = tf.nn.softmax(score, axis=1)
        context = tf.reduce_sum(weights * inputs, axis=1)
        return context

inputs = tf.keras.Input(shape=(MAX_LEN_LSTM,), dtype='int32')
x = layers.Embedding(vocab_size, 128, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
x = AttentionLayer()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(df['label'].nunique(), activation='softmax', dtype='float32')(x)
model_lstm = tf.keras.Model(inputs, outputs)

model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_lstm.summary()

t0 = time.time()
model_lstm.fit(train_seq, y_train, validation_data=(val_seq, y_val), epochs=2, batch_size=64, verbose=1)
t1 = time.time()
print("BiLSTM time (s):", t1-t0)

pred = np.argmax(model_lstm.predict(test_seq), axis=1)
print("BiLSTM+Attention results:")
compute_metrics(y_test, pred)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'attention_layer_1' (of type AttentionLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 50, 128)   │  2,217,344 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 50)        │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 50, 256)   │    263,168 │ embedding_1[0][0… │
│ (Bidirectional)     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_layer_1   │ (None, 256)       │        257 │ bidirectional_3[… │
│ (AttentionLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     32,896 │ attention_layer_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 128)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3)         │        387 │ dropout_5[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,514,052 (9.59 MB)

 Trainable params: 2,514,052 (9.59 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
272/272 ━━━━━━━━━━━━━━━━━━━━ 28s 58ms/step - accuracy: 0.8014 - loss: 0.6292 - val_accuracy: 0.8939 - val_loss: 0.3105
Epoch 2/2
272/272 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.9165 - loss: 0.2641 - val_accuracy: 0.8919 - val_loss: 0.2966
BiLSTM time (s): 37.980591773986816
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
BiLSTM+Attention results:
Accuracy: 0.9029655033286262 Macro-F1: 0.7179720446745682
              precision    recall  f1-score   support

           0       0.47      0.27      0.34       286
           1       0.93      0.95      0.94      3838
           2       0.85      0.89      0.87       833

    accuracy                           0.90      4957
   macro avg       0.75      0.70      0.72      4957
weighted avg       0.89      0.90      0.90      4957



(0.9029655033286262, 0.7179720446745682)

# **DistilBERT**

In [ ]:
train_small = train_df.sample(frac=SAMPLE_FRAC, random_state=SEED).reset_index(drop=True)
val_small   = val_df.sample(frac=SAMPLE_FRAC, random_state=SEED).reset_index(drop=True)

distil_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def encode(texts):
    enc = distil_tokenizer(list(texts), padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors='np')
    return enc['input_ids'], enc['attention_mask']

tr_ids, tr_mask = encode(train_small['text_tf'])
vl_ids, vl_mask = encode(val_small['text_tf'])
te_ids, te_mask = encode(test_df['text_tf'])

train_ds = tf.data.Dataset.from_tensor_slices(((tr_ids, tr_mask), train_small['label'].values)).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO)
val_ds   = tf.data.Dataset.from_tensor_slices(((vl_ids, vl_mask), val_small['label'].values)).batch(BATCH_SIZE).prefetch(AUTO)
test_ds  = tf.data.Dataset.from_tensor_slices(((te_ids, te_mask), test_df['label'].values)).batch(BATCH_SIZE)

def map_fn(inputs, label):
    ids, mask = inputs
    return {"input_ids": ids, "attention_mask": mask}, label

train_ds = train_ds.map(map_fn, num_parallel_calls=AUTO)
val_ds   = val_ds.map(map_fn, num_parallel_calls=AUTO)
test_ds  = test_ds.map(map_fn, num_parallel_calls=AUTO)

model_distil = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=df['label'].nunique(), use_safetensors=False)

steps_per_epoch = math.ceil(len(train_small)/BATCH_SIZE)
num_train_steps = steps_per_epoch * EPOCHS
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=int(0.06*num_train_steps), num_train_steps=num_train_steps)

model_distil.compile(optimizer=optimizer, loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model_distil.summary()

t0 = time.time()
model_distil.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)
t1 = time.time()
print("DistilBERT time (s):", t1-t0)

pred_logits = model_distil.predict(test_ds).logits
preds = np.argmax(pred_logits, axis=1)
print("DistilBERT results:")
compute_metrics(test_df['label'].values, preds)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_224']
You should probably TRAIN this model on a down-stream task to be able to use 

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
 dropout_224 (Dropout)       multiple                  0 (unused)
                                                                 
Total params: 66955779 (255.42 MB)
Trainable params: 66955779 (255.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/6
1085/1085 [=======================

(0.9150695985475086, 0.7665494037904854)

In [ ]:
from transformers import TFBertModel

bert_model = TFBertModel.from_pretrained("bert-base-uncased", use_safetensors=False)
input_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
att_mask  = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
def bert_call(inputs):
    ids, mask = inputs
    outputs = bert_model(ids, attention_mask=mask, return_dict=False)
    return outputs[0]
bert_out = tf.keras.layers.Lambda(bert_call, output_shape=(MAX_LEN, 768))([input_ids, att_mask])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/536M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training

# SVM+XGBoost

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=30000)
X_train = tfidf.fit_transform(train_df['text_cl'])
X_val   = tfidf.transform(val_df['text_cl'])
X_test  = tfidf.transform(test_df['text_cl'])

svm = LinearSVC()
xgbc = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', use_label_encoder=False, learning_rate=0.1, max_depth=6, n_estimators=200)

print("Training SVM...")
svm.fit(X_train, train_df['label'])
print("Training XGBoost...")
xgbc.fit(X_train, train_df['label'])

ensemble = VotingClassifier(estimators=[('svm', svm), ('xgb', xgbc)], voting='hard')
ensemble.fit(X_train, train_df['label'])

pred_ens = ensemble.predict(X_test)
print("Ensemble Results:")
compute_metrics(test_df['label'].values, pred_ens)


Training SVM...
Training XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:14:51] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:16:07] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Ensemble Results:
Accuracy: 0.8967117207988703 Macro-F1: 0.724191405776252
              precision    recall  f1-score   support

           0       0.53      0.32      0.40       286
           1       0.92      0.96      0.94      3838
           2       0.87      0.80      0.84       833

    accuracy                           0.90      4957
   macro avg       0.77      0.69      0.72      4957
weighted avg       0.89      0.90      0.89      4957



(0.8967117207988703, 0.724191405776252)

In [ ]:
train_small = train_df.sample(frac=SAMPLE_FRAC, random_state=SEED).reset_index(drop=True)
val_small   = val_df.sample(frac=SAMPLE_FRAC, random_state=SEED).reset_index(drop=True)

print("train_small size:", len(train_small))
print("val_small size:", len(val_small))

train_small size: 17347
val_small size: 2479


In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def encode_bert(texts):
    enc = bert_tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="np"
    )
    return enc["input_ids"], enc["attention_mask"]
tr_ids_b, tr_mask_b = encode_bert(train_small['text_tf'])
vl_ids_b, vl_mask_b = encode_bert(val_small['text_tf'])
te_ids_b, te_mask_b = encode_bert(test_df['text_tf'])

train_ds_b = tf.data.Dataset.from_tensor_slices(
    ((tr_ids_b, tr_mask_b), train_small['label'].values)
).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO)

val_ds_b = tf.data.Dataset.from_tensor_slices(
    ((vl_ids_b, vl_mask_b), val_small['label'].values)
).batch(BATCH_SIZE).prefetch(AUTO)

test_ds_b = tf.data.Dataset.from_tensor_slices(
    ((te_ids_b, te_mask_b), test_df['label'].values)
).batch(BATCH_SIZE).prefetch(AUTO)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# **BERT+CNN**

In [ ]:
from transformers import TFBertModel

bert_model_cnn = TFBertModel.from_pretrained("bert-base-uncased", use_safetensors=False)

input_ids_c = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids_c")
att_mask_c  = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask_c")

def bert_call_c(inputs):
    ids, mask = inputs
    outputs = bert_model_cnn(ids, attention_mask=mask, return_dict=False)
    return outputs[0]

bert_out_c = tf.keras.layers.Lambda(
    bert_call_c,
    output_shape=(MAX_LEN, 768)
)([input_ids_c, att_mask_c])

c1 = tf.keras.layers.Conv1D(128, 3, activation="relu")(bert_out_c)
c1 = tf.keras.layers.GlobalMaxPool1D()(c1)

c2 = tf.keras.layers.Conv1D(128, 4, activation="relu")(bert_out_c)
c2 = tf.keras.layers.GlobalMaxPool1D()(c2)

c3 = tf.keras.layers.Conv1D(128, 5, activation="relu")(bert_out_c)
c3 = tf.keras.layers.GlobalMaxPool1D()(c3)

concat = tf.keras.layers.Concatenate()([c1, c2, c3])
drop   = tf.keras.layers.Dropout(0.3)(concat)

num_labels = df['label'].nunique()

out_c  = tf.keras.layers.Dense(num_labels, activation="softmax", dtype="float32")(drop)

model_bert_cnn = tf.keras.Model(
    inputs=[input_ids_c, att_mask_c],
    outputs=out_c
)
steps_per_epoch_bert_cnn = math.ceil(len(train_df)/BATCH_SIZE)
num_train_steps_bert_cnn = steps_per_epoch_bert_cnn * EPOCHS

initial_learning_rate_cnn = 2e-5
warmup_steps_cnn = int(0.06 * num_train_steps_bert_cnn)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, warmup_steps, total_steps):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)
        self.total_steps = tf.cast(total_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        lr = tf.where(
            step < self.warmup_steps,
            self.initial_learning_rate * (step / self.warmup_steps),
            self.initial_learning_rate * (1.0 - (step - self.warmup_steps) / (self.total_steps - self.warmup_steps))
        )
        return tf.maximum(0.0, lr)

lr_schedule_cnn = CustomSchedule(initial_learning_rate_cnn, warmup_steps_cnn, num_train_steps_bert_cnn)
optimizer_keras_cnn = tf.keras.optimizers.Adam(learning_rate=lr_schedule_cnn)

model_bert_cnn.compile(
    optimizer=optimizer_keras_cnn,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model_bert_cnn.summary()
def ds_mapper_cnn(features, label):
    return {
        "input_ids_c": features[0],
        "attention_mask_c": features[1]
    }, label

train_c = train_ds_b.map(ds_mapper_cnn)
val_c   = val_ds_b.map(ds_mapper_cnn)
test_c  = test_ds_b.map(ds_mapper_cnn)
model_bert_cnn.fit(train_c, validation_data=val_c, epochs=EPOCHS)
pred_c = np.argmax(model_bert_cnn.predict(test_c), axis=1)
compute_metrics(test_df["label"].values, pred_c)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids_c         │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask_c    │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_6 (Lambda)   │ (None, 128, 768)  │          0 │ input_ids_c[0][0… │
│                     │                   │            │ attention_mask_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 126, 128)  │    295,040 │ lambda_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 125, 128)  │    393,344 │ lambda_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 124, 128)  │    491,648 │ lambda_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_6[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_7[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_8[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 384)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 384)       │          0 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 3)         │      1,155 │ dropout_6[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,181,187 (4.51 MB)

 Trainable params: 1,181,187 (4.51 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 185s 151ms/step - accuracy: 0.7387 - loss: 0.8154 - val_accuracy: 0.8463 - val_loss: 0.4191
Epoch 2/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 146s 135ms/step - accuracy: 0.8328 - loss: 0.4759 - val_accuracy: 0.8628 - val_loss: 0.3671
Epoch 3/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 146s 134ms/step - accuracy: 0.8548 - loss: 0.3921 - val_accuracy: 0.8693 - val_loss: 0.3540
Epoch 4/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 146s 135ms/step - accuracy: 0.8745 - loss: 0.3464 - val_accuracy: 0.8709 - val_loss: 0.3479
Epoch 5/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 202s 135ms/step - accuracy: 0.8860 - loss: 0.3136 - val_accuracy: 0.8737 - val_loss: 0.3410
Epoch 6/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 146s 134ms/step - accuracy: 0.8883 - loss: 0.2979 - val_accuracy: 0.8733 - val_loss: 0.3387
310/310 ━━━━━━━━━━━━━━━━━━━━ 46s 137ms/step
Accuracy: 0.8704861811579584 Macro-F1: 0.6339215516928179
              precision    recall  f1-score   support

           0       0.58      0.15      0.23  

(0.8704861811579584, 0.6339215516928179)

In [ ]:
optimizer_main = tf.keras.optimizers.Adam(learning_rate=2e-5)

# **BERT+LSTM**

In [ ]:
from transformers import TFBertModel
import tensorflow as tf
import math
bert_model_lstm = TFBertModel.from_pretrained("bert-base-uncased", use_safetensors=False)
input_ids_l = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids_l")
att_mask_l  = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask_l")
def bert_call_l(inputs):
    ids, mask = inputs
    outputs = bert_model_lstm(ids, attention_mask=mask, return_dict=False)
    return outputs[0]

bert_out_l = tf.keras.layers.Lambda(
    bert_call_l,
    output_shape=(MAX_LEN, 768)
)([input_ids_l, att_mask_l])

# LSTM layer (bidirectional)
lstm_out = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=False)
)(bert_out_l)

drop = tf.keras.layers.Dropout(0.3)(lstm_out)

num_labels = df['label'].nunique()
out_l = tf.keras.layers.Dense(num_labels, activation="softmax", dtype="float32")(drop)

model_bert_lstm = tf.keras.Model(
    inputs=[input_ids_l, att_mask_l],
    outputs=out_l
)
steps_per_epoch_lstm = math.ceil(len(train_df) / BATCH_SIZE)
num_train_steps_lstm = steps_per_epoch_lstm * EPOCHS

initial_lr_lstm = 2e-5
warmup_steps_lstm = int(0.06 * num_train_steps_lstm)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, warmup_steps, total_steps):
        super().__init__()
        self.initial_lr = initial_lr
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)
        self.total_steps = tf.cast(total_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        lr = tf.where(
            step < self.warmup_steps,
            self.initial_lr * (step / self.warmup_steps),
            self.initial_lr * (1.0 - (step - self.warmup_steps) / (self.total_steps - self.warmup_steps))
        )
        return tf.maximum(0.0, lr)

lr_schedule_lstm = CustomSchedule(initial_lr_lstm, warmup_steps_lstm, num_train_steps_lstm)
optimizer_lstm = tf.keras.optimizers.Adam(learning_rate=lr_schedule_lstm)

model_bert_lstm.compile(
    optimizer=optimizer_lstm,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model_bert_lstm.summary()

def ds_mapper_lstm(features, label):
    return {
        "input_ids_l": features[0],
        "attention_mask_l": features[1]
    }, label

train_l = train_ds_b.map(ds_mapper_lstm)
val_l   = val_ds_b.map(ds_mapper_lstm)
test_l  = test_ds_b.map(ds_mapper_lstm)
model_bert_lstm.fit(train_l, validation_data=val_l, epochs=EPOCHS)
pred_l = np.argmax(model_bert_lstm.predict(test_l), axis=1)
compute_metrics(test_df["label"].values, pred_l)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids_l         │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask_l    │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 128, 768)  │          0 │ input_ids_l[0][0… │
│                     │                   │            │ attention_mask_l… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 512)       │  2,099,200 │ lambda_1[0][0]    │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3)         │      1,539 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,100,739 (8.01 MB)

 Trainable params: 2,100,739 (8.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 375s 251ms/step - accuracy: 0.6792 - loss: 0.7458 - val_accuracy: 0.8467 - val_loss: 0.4182
Epoch 2/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 218s 201ms/step - accuracy: 0.8552 - loss: 0.4081 - val_accuracy: 0.8641 - val_loss: 0.3803
Epoch 3/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 198s 182ms/step - accuracy: 0.8683 - loss: 0.3697 - val_accuracy: 0.8665 - val_loss: 0.3728
Epoch 4/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 196s 181ms/step - accuracy: 0.8779 - loss: 0.3466 - val_accuracy: 0.8661 - val_loss: 0.3649
Epoch 5/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 196s 181ms/step - accuracy: 0.8799 - loss: 0.3366 - val_accuracy: 0.8685 - val_loss: 0.3611
Epoch 6/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 196s 181ms/step - accuracy: 0.8910 - loss: 0.3146 - val_accuracy: 0.8729 - val_loss: 0.3599
310/310 ━━━━━━━━━━━━━━━━━━━━ 47s 147ms/step
Accuracy: 0.866653217671979 Macro-F1: 0.613323892518472
              precision    recall  f1-score   support

           0       0.44      0.11      0.17    

(0.866653217671979, 0.613323892518472)

# **BERT+GRU**

In [ ]:
from transformers import TFBertModel
import tensorflow as tf
import math
bert_model_gru = TFBertModel.from_pretrained("bert-base-uncased", use_safetensors=False)
input_ids_g = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids_g")
att_mask_g  = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask_g")
def bert_call_g(inputs):
    ids, mask = inputs
    outputs = bert_model_gru(ids, attention_mask=mask, return_dict=False)
    return outputs[0]

bert_out_g = tf.keras.layers.Lambda(
    bert_call_g,
    output_shape=(MAX_LEN, 768)
)([input_ids_g, att_mask_g])
gru_out = tf.keras.layers.Bidirectional(
    tf.keras.layers.GRU(256, return_sequences=False)
)(bert_out_g)

drop = tf.keras.layers.Dropout(0.3)(gru_out)

num_labels = df['label'].nunique()
out_g = tf.keras.layers.Dense(num_labels, activation="softmax", dtype="float32")(drop)

model_bert_gru = tf.keras.Model(
    inputs=[input_ids_g, att_mask_g],
    outputs=out_g
)
steps_per_epoch_gru = math.ceil(len(train_df) / BATCH_SIZE)
num_train_steps_gru = steps_per_epoch_gru * EPOCHS

initial_lr_gru = 2e-5
warmup_steps_gru = int(0.06 * num_train_steps_gru)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, warmup_steps, total_steps):
        super().__init__()
        self.initial_lr = initial_lr
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)
        self.total_steps = tf.cast(total_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        lr = tf.where(
            step < self.warmup_steps,
            self.initial_lr * (step / self.warmup_steps),
            self.initial_lr * (1.0 - (step - self.warmup_steps) / (self.total_steps - self.warmup_steps))
        )
        return tf.maximum(0.0, lr)

lr_schedule_gru = CustomSchedule(initial_lr_gru, warmup_steps_gru, num_train_steps_gru)
optimizer_gru = tf.keras.optimizers.Adam(learning_rate=lr_schedule_gru)

model_bert_gru.compile(
    optimizer=optimizer_gru,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model_bert_gru.summary()
def ds_mapper_gru(features, label):
    return {
        "input_ids_g": features[0],
        "attention_mask_g": features[1]
    }, label

train_g = train_ds_b.map(ds_mapper_gru)
val_g   = val_ds_b.map(ds_mapper_gru)
test_g  = test_ds_b.map(ds_mapper_gru)
model_bert_gru.fit(train_g, validation_data=val_g, epochs=EPOCHS)
pred_g = np.argmax(model_bert_gru.predict(test_g), axis=1)
compute_metrics(test_df["label"].values, pred_g)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids_g         │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask_g    │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_8 (Lambda)   │ (None, 128, 768)  │          0 │ input_ids_g[0][0… │
│                     │                   │            │ attention_mask_g… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_5     │ (None, 512)       │  1,575,936 │ lambda_8[0][0]    │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 512)       │          0 │ bidirectional_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 3)         │      1,539 │ dropout_8[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,577,475 (6.02 MB)

 Trainable params: 1,577,475 (6.02 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 214s 186ms/step - accuracy: 0.7082 - loss: 0.7251 - val_accuracy: 0.8233 - val_loss: 0.4759
Epoch 2/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 198s 183ms/step - accuracy: 0.8278 - loss: 0.4793 - val_accuracy: 0.8447 - val_loss: 0.4318
Epoch 3/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 199s 183ms/step - accuracy: 0.8513 - loss: 0.4191 - val_accuracy: 0.8495 - val_loss: 0.4074
Epoch 4/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 199s 183ms/step - accuracy: 0.8598 - loss: 0.3936 - val_accuracy: 0.8592 - val_loss: 0.3953
Epoch 5/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 199s 183ms/step - accuracy: 0.8640 - loss: 0.3759 - val_accuracy: 0.8612 - val_loss: 0.3903
Epoch 6/6
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 198s 182ms/step - accuracy: 0.8690 - loss: 0.3687 - val_accuracy: 0.8624 - val_loss: 0.3888
310/310 ━━━━━━━━━━━━━━━━━━━━ 49s 152ms/step
Accuracy: 0.8599959653015937 Macro-F1: 0.5595433240458244
              precision    recall  f1-score   support

           0       0.40      0.02      0.04  

(0.8599959653015937, 0.5595433240458244)